In [1]:
import torch
import datasets
import os
import time
import numpy as np
import sys
import pickle
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    GenerationConfig
)

### Skip this

In [2]:
llama_causal = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import BaseQuantizeConfig
quant_config = BaseQuantizeConfig(nbits=4, group_size=64)
AutoHQQHFModel.quantize_model(llama_causal, quant_config=quant_config, 
                                    compute_dtype=torch.float16, 
                                    device="cuda")
# 1.4gb

100%|██████████| 155/155 [00:17<00:00,  9.00it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (k_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (v_proj): HQQLinear(in_features=2048, out_features=256, bias=False)
          (o_proj): HQQLinear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (up_proj): HQQLinear(in_features=2048, out_features=5632, bias=False)
          (down_proj): HQQLinear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [7]:
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.save_quantized(llama_causal, "./tinyllama_hqq_4_64")

### Continue

In [4]:
from hqq.models.hf.base import AutoHQQHFModel
llama_causal = AutoHQQHFModel.from_quantized("../artifacts/tinyllama_hqq_4_64")

100%|██████████| 155/155 [00:00<00:00, 59764.40it/s]


In [5]:
from transformers import AutoTokenizer
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [80]:
# . Here are some examples of valid responses.
#         Example 1: <user> I AM CRYING WAAAAAA!!!! I want ice cream :( <user><assistant> sad <assistant>
#         Example 2: <user> how could YOU BETRAY ME?!?!? <user><assistant> angry <assistant>
#         Example 3: <user> YAY FRIENDSHIP IS MAGIC!! <user><assistant> happy <assistant>
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are now an expert on recognizing emotions in conversations. You understand emotions to all fall into one of the following categories: [angry, happy, sad, scared, other].
        """,
    },
    {
        "role": "user",
        "content": "Classify the following based on expert knowledge: I AM CRYING WAAAAAA!!!! I want ice cream :(",
    },
    {
        "role": "assistant",
        "content": "sad",
    },
    {
        "role": "user",
        "content": "Classify the following as before: how could YOU BETRAY ME?!?!?",
    },
    {
        "role": "assistant",
        "content": "angry",
    },
    {
        "role": "user",
        "content": "Classify the following as before: euphoria is filling my eeyes or something!",
    }
], return_tensors='pt')

In [107]:
input_ids = llama_tokenizer.apply_chat_template([
    {
        "role": "system",
        "content": """You are now an expert on personalities. You generate each personalities item seperated by a newline, with no bulleting
        """,
    },
    {
        "role": "user",
        "content": "Describe a person's 5 personality traits",
    },
    {
        "role": "assistant",
        "content": "sad\npeckish\nenvious\nsarcastic\njealous",
    },
    {
        "role": "user",
        "content": "Describe a person's 5 personality traits with the previous list formatting",
    },
    {
        "role": "assistant",
        "content": "Happyg go lucky\ndepressed\nCold hearted\nfriendly\nAntagonistic",
    },
    {
        "role": "user",
        "content": "Describe a person's 5 personality traits based on expert knowledge with the previous list formatting",
    }
], return_tensors='pt')

In [108]:
gen_cfg = GenerationConfig.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
gen_cfg.max_length = 256

In [109]:
llama_tokenizer.decode(llama_causal.generate(input_ids.cuda().long(), generation_config=gen_cfg)[0])

"<|system|>\nYou are now an expert on personalities. You generate each personalities item seperated by a newline, with no bulleting\n        </s> \n<|user|>\nDescribe a person's 5 personality traits</s> \n<|assistant|>\nsad\npeckish\nenvious\nsarcastic\njealous</s> \n<|user|>\nDescribe a person's 5 personality traits with the previous list formatting</s> \n<|assistant|>\nHappyg go lucky\ndepressed\nCold hearted\nfriendly\nAntagonistic</s> \n<|user|>\nDescribe a person's 5 personality traits based on expert knowledge with the previous list formatting</s> \n<|assistant|>\nHappyg go lucky\nCold hearted\nfriendly\nAntagonistic\nsad\npeckish\nenvious\nsarcastic\njealous</s>"